<a href="https://colab.research.google.com/github/orenshor/eCommerce_project/blob/master/task_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Embedding, Input, Dense, Flatten, Dropout
from keras.optimizers import Adam, Adamax
from keras.layers import Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [0]:
RATING_DATA_TEST_FILE = "u1.test"
RATING_DATA_TRAIN_FILE = "u1.base"
MODEL_WEIGHTS_FILE = "u_emb_weights.h5"

USER_DATA_FILE = 'u.user'
MODEL_WEIGHTS_FILE_CORE = 'u_emb_weights'

In [0]:
#import of the data

user_cols = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
movie_cols = ['user_id','movie_id','rating','timestamp']

# load users data
user_data = pd.read_csv(USER_DATA_FILE, sep='|', engine='python', encoding='latin-1', names=user_cols)

# replace values in the data
user_data['gender'].replace(['F','M'],['0','1'],inplace=True)
user_data['age'] = pd.cut(user_data['age'],bins=[0,18,25,30,40,50,100], labels=["0","1","2","3","4","5"])

# load movies data
ratings_train = pd.read_csv(RATING_DATA_TRAIN_FILE, sep='\t', engine='python', encoding='latin-1', names=movie_cols)

# marge users info and ratings 
ratings_train = pd.merge(ratings_train, user_data, on='user_id',how='inner')
print(ratings_train.head())
# train
max_userid = ratings_train['user_id'].drop_duplicates().max()
max_movieid = ratings_train['movie_id'].drop_duplicates().max()
ratings_train['user_emb_id'] = ratings_train['user_id'] - 1
ratings_train['movie_emb_id'] = ratings_train['movie_id'] - 1
print(str(len(ratings_train))+' ratings loaded from train')

# test
ratings_test = pd.read_csv(RATING_DATA_TEST_FILE, sep='\t',   engine='python', encoding='latin-1',  names=movie_cols)
ratings_test = pd.merge(ratings_test, user_data, on='user_id', how='inner')
print(ratings_test.head())
ratings_test['user_emb_id'] = ratings_test['user_id'] - 1
ratings_test['movie_emb_id'] = ratings_test['movie_id'] - 1
print(str(len(ratings_test))+' ratings loaded from test')



In [0]:
# train values
train_users = ratings_train['user_emb_id'].values
train_movies = ratings_train['movie_emb_id'].values
train_ratings = ratings_train['rating'].values
train_gender = ratings_train['gender'].values
train_age = ratings_train['age'].values

Test_Users = ratings_test['user_emb_id'].values
Test_Movies = ratings_test['movie_emb_id'].values
Test_Ratings = ratings_test['rating'].values

In [0]:
# model from ex. 4 with adjustments
def get_ncf_model_gen1(num_users, num_items, latent_dim, hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    gender_input = Input(shape=(1,), dtype='float32', name = 'gender_input')
    age_input = Input(shape=(1,), dtype='float32', name = 'age_input')

    NCF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    NCF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(NCF_Embedding_User(user_input))
    item_latent = Flatten()(NCF_Embedding_Item(item_input))
    
    # Concat user and item embeddings with gender
    conc = Concatenate()([user_latent, item_latent, gender_input, age_input])
    drop = Dropout(do)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    model = Model(inputs=[user_input, item_input, gender_input, age_input], outputs=prediction)
    print("ncf model gender 1")
    model.summary()

    return model

In [0]:
K_LATENT = 20
hidden_dim = 20
do = 0.3

NCF_G_model1 = get_ncf_model_gen1(max_userid, max_movieid, K_LATENT,hidden_dim, do)
NCF_G_model1.compile(loss='mse',optimizer=Adamax(),metrics=['mae'])

In [99]:
callbacks_ncfg1 = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE + '_ncfg_' + str(do) + '_' + str(K_LATENT) + '.h5', save_best_only=True)]
history_history_ncfg1 = NCF_G_model1.fit([train_users, train_movies, train_gender, train_age], train_ratings, epochs=100, validation_split=.1, verbose=1, callbacks=callbacks_ncfg1, batch_size = 32)

Train on 72000 samples, validate on 8000 samples
Epoch 1/100
72000/72000 [==============================] - 4s 53us/step - loss: 0.8743 - mae: 0.7386 - val_loss: 1.0509 - val_mae: 0.8358
Epoch 2/100
72000/72000 [==============================] - 4s 50us/step - loss: 0.8632 - mae: 0.7327 - val_loss: 1.0419 - val_mae: 0.8335
Epoch 3/100
72000/72000 [==============================] - 4s 50us/step - loss: 0.8567 - mae: 0.7296 - val_loss: 1.0272 - val_mae: 0.8196
Epoch 4/100
72000/72000 [==============================] - 4s 51us/step - loss: 0.8542 - mae: 0.7289 - val_loss: 1.0326 - val_mae: 0.8298
Epoch 5/100
72000/72000 [==============================] - 4s 52us/step - loss: 0.8464 - mae: 0.7246 - val_loss: 1.0583 - val_mae: 0.8444
Epoch 6/100
72000/72000 [==============================] - 4s 52us/step - loss: 0.8445 - mae: 0.7236 - val_loss: 1.0335 - val_mae: 0.8305
Epoch 7/100
72000/72000 [==============================] - 4s 61us/step - loss: 0.8411 - mae: 0.7222 - val_loss: 1.0344 - v

In [100]:
# test values
test_users = ratings_test['user_emb_id'].values
test_movies = ratings_test['movie_emb_id'].values
test_ratings = ratings_test['rating'].values
test_gender = ratings_test['gender'].values
test_age = ratings_test['age'].values

preddict_model_gen1 = NCF_G_model1.predict([test_users,test_movies, test_gender, test_age])
test_predict1 = pd.DataFrame(data=preddict_model_gen1, columns=['prediction'])
test_predict1['actual_rating'] = test_ratings

MAE1 = np.sum(abs(test_predict1['actual_rating']-test_predict1['prediction']))/test_predict1.shape[0]
print("MAE = "+ str(MAE1))

MAE = 0.84877578125


In [0]:
pip install deepctr[cpu]

In [0]:
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, get_feature_names
from sklearn.preprocessing import LabelEncoder

In [102]:
for feat in ['user_emb_id','movie_emb_id', 'gender', 'age']:
        lbe = LabelEncoder()
        ratings_train[feat] = lbe.fit_transform(ratings_train[feat])


for feat in ['user_emb_id','movie_emb_id', 'gender', 'age']:
        lbe = LabelEncoder()
        ratings_test[feat] = lbe.fit_transform(ratings_test[feat])


fixlen_feature_columns = [SparseFeat(feat, ratings_train[feat].nunique(), embedding_dim=20)
                              for feat in ['user_emb_id','movie_emb_id', 'gender', 'age']]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(feature_names)


model_deepFM = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model_deepFM.compile("adam", "mse", metrics=['mae'])

train_model_deepFM_input = {name: ratings_train[name].values for name in feature_names}

test_model_deepFM_input = {name: ratings_test[name].values for name in feature_names}

history = model_deepFM.fit(train_model_deepFM_input, train_ratings, epochs=100,  verbose=1, validation_split=0.1)

pred_deepFM = model_deepFM.predict(test_model_deepFM_input)
test_predict_deepFM = pd.DataFrame(data=pred_deepFM, columns=['prediction'])
test_predict_deepFM['actual_rating'] = test_ratings
MAE_deepFM = np.sum(abs(test_predict_deepFM['actual_rating']-test_predict_deepFM['prediction']))/test_predict_deepFM.shape[0]
print()
print("MAE deepFM = " + str(MAE_deepFM))

['user_emb_id', 'movie_emb_id', 'gender', 'age']
Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2250/2250 [==============================] - 7s 3ms/step - loss: 1.0290 - mae: 0.8008 - val_loss: 1.0336 - val_mae: 0.8355
Epoch 2/100
2250/2250 [==============================] - 6s 3ms/step - loss: 0.8843 - mae: 0.7434 - val_loss: 1.0127 - val_mae: 0.8147
Epoch 3/100
2250/2250 [==============================] - 7s 3ms/step - loss: 0.8518 - mae: 0.7280 - val_loss: 0.9665 - val_mae: 0.7861
Epoch 4/100
2250/2250 [==============================] - 6s 3ms/step - loss: 0.8220 - mae: 0.7125 - val_loss: 0.9953 - val_mae: 0.8049
Epoch 5/100
2250/2250 [==============================] - 6s 3ms/step - loss: 0.7904 - mae: 0.6966 - val_loss: 1.0197 - val_mae: 0.8168
Epoch 6/100
2250/2250 [==============================] - 7s 3ms/step - loss: 0.7599 - mae: 0.6810 - val_loss: 0.9753 - val_mae: 0.7871
Epoch 7/100
2250/2250 [==============================] - 6s 3ms/step - loss: 0.7287 - mae: 0.6650 - val_loss: 0.9899 - val_mae: 0.7993
Epoch 8/100
2250/2250 [==============================] - 6s 3ms/ste

In [0]:
def get_ncf_model_gen2(num_users, num_items, latent_dim, hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    gender_input = Input(shape=(1,), dtype='float32', name = 'gender_input')
    age_input = Input(shape=(1,), dtype='float32', name = 'age_input')

    NCF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    NCF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(NCF_Embedding_User(user_input))
    item_latent = Flatten()(NCF_Embedding_Item(item_input))
    
    # Concat user and item embeddings with gender
    conc = Concatenate()([user_latent, item_latent, gender_input, age_input])
    drop = Dropout(0.3)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    hid2 = Dense(hidden_dim,activation='relu')(drop2)
    drop3  = Dropout(do)(hid2)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop3)
    
    model = DeepFM(inputs=[user_input, item_input, gender_input, age_input], outputs=prediction)
    print("ncf model gender 2")
    model.summary()

    return model

In [60]:
K_LATENT = 20
hidden_dim = 20
do = 0.5
NCF_G_model2 = get_ncf_model_gen2(max_userid, max_movieid, K_LATENT, hidden_dim, do)
NCF_G_model2.compile(loss='mse',optimizer=Adamax(),metrics=['mae'])

TypeError: ignored

In [0]:
callbacks_ncfg2 = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE + '_ncfg_' + str(do)+'_' + str(K_LATENT) + '.h5', save_best_only=True)]
history_history_ncfg2 = NCF_G_model2.fit([train_users, train_movies, train_gender, train_age], train_ratings, epochs=100, 
                                         validation_split=.1, verbose=1, callbacks=callbacks_ncfg2, batch_size = 32)

In [46]:
preddict_model_gen2 = NCF_G_model2.predict([test_users,test_movies, test_gender, test_age])
test_predict2 = pd.DataFrame(data=preddict_model_gen2, columns=['prediction'])
test_predict2['actual_rating'] = test_ratings

MAE2 = np.sum(abs(test_predict2['actual_rating']-test_predict2['prediction']))/test_predict2.shape[0]
print("MAE = "+ str(MAE2))

MAE = 0.7457966796875
